In [1]:
import pandas as pd
import copy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.cluster import MiniBatchKMeans

from sklearn.metrics import recall_score, precision_score
#from sklearn.cross_validation import KFold, train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
%matplotlib inline

In [2]:
young = pd.read_csv('input/responses.csv')

In [3]:
young.head(10).columns[:19]

Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       'Punk', 'Hiphop, Rap', 'Reggae, Ska', 'Swing, Jazz', 'Rock n roll',
       'Alternative', 'Latino', 'Techno, Trance', 'Opera'],
      dtype='object')

In [25]:
young.iloc[:,:19].describe()

Music  Slow songs or fast songs        Dance         Folk  \
count  1007.000000               1008.000000  1006.000000  1005.000000   
mean      4.731877                  3.328373     3.113320     2.288557   
std       0.664049                  0.833931     1.170568     1.138916   
min       1.000000                  1.000000     1.000000     1.000000   
25%       5.000000                  3.000000     2.000000     1.000000   
50%       5.000000                  3.000000     3.000000     2.000000   
75%       5.000000                  4.000000     4.000000     3.000000   
max       5.000000                  5.000000     5.000000     5.000000   

           Country  Classical music      Musical          Pop         Rock  \
count  1005.000000      1003.000000  1008.000000  1007.000000  1004.000000   
mean      2.123383         2.956132     2.761905     3.471698     3.761952   
std       1.076136         1.252570     1.260845     1.161400     1.184861   
min       1.000000         1.000000     1.000000     1.000000     1.000000   
25%       1.000000         2.000000     2.000000     3.000000     3.000000   
50%       2.000000         3.000000     3.000000     4.000000     4.000000   
75%       3.000000         4.000000     4.000000     4.000000     5.000000   
max       5.000000         5.000000     5.000000     5.000000     5.000000   

       Metal or Hardrock         Punk  Hiphop, Rap  Reggae, Ska  Swing, Jazz  \
count        1007.000000  1002.000000  1006.000000  1003.000000  1004.000000   
mean            2.361470     2.456088     2.910537     2.769691     2.759960   
std             1.372995     1.301105     1.375677     1.214434     1.257936   
min             1.000000     1.000000     1.000000     1.000000     1.000000   
25%             1.000000     1.000000     2.000000     2.000000     2.000000   
50%             2.000000     2.000000     3.000000     3.000000     3.000000   
75%             3.000000     3.000000     4.000000     4.000000     4.000000   
max             5.000000     5.000000     5.000000     5.000000     5.000000   

       Rock n roll  Alternative       Latino  Techno, Trance        Opera  
count  1003.000000  1003.000000  1002.000000     1003.000000  1009.000000  
mean      3.141575     2.828514     2.842315        2.338983     2.139742  
std       1.237269     1.347173     1.327902        1.324099     1.184094  
min       1.000000     1.000000     1.000000        1.000000     1.000000  
25%       2.000000     2.000000     2.000000        1.000000     1.000000  
50%       3.000000     3.000000     3.000000        2.000000     2.000000  
75%       4.000000     4.000000     4.000000        3.000000     3.000000  
max       5.000000     5.000000     5.000000        5.000000     5.000000

In [6]:
young_music_full = young.dropna(axis=0)#.iloc[:,:19]

In [48]:
young_music_full

Music  Slow songs or fast songs  Dance  Folk  Country  Classical music  \
0       5.0                       3.0    2.0   1.0      2.0              2.0   
1       4.0                       4.0    2.0   1.0      1.0              1.0   
2       5.0                       5.0    2.0   2.0      3.0              4.0   
4       5.0                       3.0    4.0   3.0      2.0              4.0   
5       5.0                       3.0    2.0   3.0      2.0              3.0   
6       5.0                       5.0    5.0   3.0      1.0              2.0   
7       5.0                       3.0    3.0   2.0      1.0              2.0   
9       5.0                       3.0    2.0   5.0      2.0              2.0   
10      5.0                       3.0    3.0   2.0      1.0              2.0   
11      5.0                       3.0    1.0   1.0      1.0              4.0   
12      5.0                       3.0    1.0   2.0      1.0              4.0   
13      5.0                       3.0    5.0   3.0      2.0              1.0   
14      5.0                       3.0    2.0   1.0      1.0              2.0   
16      5.0                       3.0    3.0   1.0      1.0              1.0   
18      5.0                       3.0    5.0   4.0      3.0              4.0   
19      5.0                       4.0    3.0   3.0      2.0              4.0   
20      5.0                       3.0    3.0   2.0      3.0              4.0   
21      5.0                       5.0    1.0   1.0      3.0              2.0   
23      5.0                       3.0    4.0   2.0      2.0              2.0   
24      5.0                       2.0    3.0   1.0      1.0              4.0   
25      5.0                       3.0    4.0   2.0      1.0              2.0   
26      5.0                       5.0    5.0   5.0      4.0              5.0   
28      5.0                       3.0    5.0   1.0      1.0              1.0   
29      5.0                       4.0    3.0   4.0      2.0              3.0   
30      4.0                       3.0    4.0   3.0      3.0              3.0   
31      4.0                       3.0    4.0   1.0      3.0              2.0   
32      5.0                       5.0    3.0   1.0      3.0              2.0   
33      5.0                       4.0    2.0   2.0      3.0              4.0   
34      5.0                       4.0    3.0   2.0      1.0              3.0   
35      5.0                       3.0    3.0   3.0      1.0              4.0   
...     ...                       ...    ...   ...      ...              ...   
966     5.0                       5.0    1.0   1.0      1.0              5.0   
967     5.0                       3.0    2.0   2.0      2.0              3.0   
969     5.0                       3.0    2.0   1.0      2.0              3.0   
970     5.0                       3.0    5.0   1.0      4.0              5.0   
971     5.0                       4.0    2.0   4.0      3.0              4.0   
972     5.0                       3.0    3.0   3.0      3.0              3.0   
973     5.0                       3.0    2.0   5.0      5.0              4.0   
974     5.0                       3.0    1.0   2.0      2.0              2.0   
976     5.0                       3.0    4.0   2.0      3.0              3.0   
977     5.0                       4.0    4.0   2.0      2.0              1.0   
978     5.0                       2.0    2.0   1.0      2.0              2.0   
979     5.0                       3.0    4.0   2.0      2.0              4.0   
981     5.0                       4.0    1.0   2.0      4.0              5.0   
982     5.0                       4.0    2.0   1.0      1.0              3.0   
985     5.0                       2.0    1.0   3.0      2.0              4.0   
986     4.0                       3.0    2.0   5.0      3.0              5.0   
989     5.0                       5.0    3.0   2.0      2.0              4.0   
990     5.0                       4.0    2.0   1.0      1.0              2.0   
991   

In [7]:
young_music_full.shape#[:1]

(674, 150)

In [8]:
X=np.array(young_music_full.iloc[:,:19])

In [9]:
X.shape

(674, 19)

In [10]:
kmeans = MiniBatchKMeans(n_clusters=3,
...                          random_state=0,
...                          batch_size=6,
...                          max_iter=10).fit(X)

In [53]:
kmeans.cluster_centers_

array([[4.96153846, 3.01923077, 3.48076923, 3.59615385, 2.96153846,
        4.23076923, 4.05769231, 3.69230769, 3.92307692, 2.26923077,
        2.21153846, 3.17307692, 2.98076923, 4.11538462, 3.75      ,
        3.21153846, 3.96153846, 2.48076923, 3.51923077],
       [4.79411765, 3.22058824, 2.20588235, 2.86764706, 2.55882353,
        3.97058824, 3.35294118, 2.32352941, 4.70588235, 3.63235294,
        3.39705882, 1.73529412, 2.88235294, 3.30882353, 4.01470588,
        3.72058824, 2.60294118, 1.41176471, 3.22058824],
       [4.7625    , 3.5       , 3.2875    , 1.6125    , 1.425     ,
        1.85      , 1.725     , 3.775     , 2.7625    , 1.4       ,
        1.5875    , 3.2875    , 2.1       , 1.5125    , 1.9375    ,
        1.8875    , 1.975     , 2.4625    , 1.225     ],
       [4.69421488, 3.29752066, 3.52066116, 2.        , 1.97520661,
        2.90082645, 2.97520661, 3.80165289, 3.52066116, 1.80165289,
        2.07438017, 3.40495868, 3.01652893, 3.05785124, 2.96694215,
        2.578

In [11]:
predicted_music_group = kmeans.predict(X)

In [12]:
predicted_music_group

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 2, 2, 2, 2,
       1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 2,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 2, 1,
       2, 0, 2, 0, 2, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 2, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 2, 0, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 2,
       0, 1, 0, 2, 1, 0, 0, 2, 0, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 2, 0,

In [13]:
young_music_full['cluster']=predicted_music_group

/Users/thalia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
young_music_full#.shape

Music  Slow songs or fast songs  Dance  Folk  Country  Classical music  \
0       5.0                       3.0    2.0   1.0      2.0              2.0   
1       4.0                       4.0    2.0   1.0      1.0              1.0   
2       5.0                       5.0    2.0   2.0      3.0              4.0   
4       5.0                       3.0    4.0   3.0      2.0              4.0   
5       5.0                       3.0    2.0   3.0      2.0              3.0   
6       5.0                       5.0    5.0   3.0      1.0              2.0   
7       5.0                       3.0    3.0   2.0      1.0              2.0   
9       5.0                       3.0    2.0   5.0      2.0              2.0   
10      5.0                       3.0    3.0   2.0      1.0              2.0   
11      5.0                       3.0    1.0   1.0      1.0              4.0   
12      5.0                       3.0    1.0   2.0      1.0              4.0   
13      5.0                       3.0    5.0   3.0      2.0              1.0   
14      5.0                       3.0    2.0   1.0      1.0              2.0   
16      5.0                       3.0    3.0   1.0      1.0              1.0   
18      5.0                       3.0    5.0   4.0      3.0              4.0   
19      5.0                       4.0    3.0   3.0      2.0              4.0   
20      5.0                       3.0    3.0   2.0      3.0              4.0   
21      5.0                       5.0    1.0   1.0      3.0              2.0   
23      5.0                       3.0    4.0   2.0      2.0              2.0   
24      5.0                       2.0    3.0   1.0      1.0              4.0   
25      5.0                       3.0    4.0   2.0      1.0              2.0   
26      5.0                       5.0    5.0   5.0      4.0              5.0   
28      5.0                       3.0    5.0   1.0      1.0              1.0   
29      5.0                       4.0    3.0   4.0      2.0              3.0   
30      4.0                       3.0    4.0   3.0      3.0              3.0   
31      4.0                       3.0    4.0   1.0      3.0              2.0   
32      5.0                       5.0    3.0   1.0      3.0              2.0   
33      5.0                       4.0    2.0   2.0      3.0              4.0   
34      5.0                       4.0    3.0   2.0      1.0              3.0   
35      5.0                       3.0    3.0   3.0      1.0              4.0   
...     ...                       ...    ...   ...      ...              ...   
966     5.0                       5.0    1.0   1.0      1.0              5.0   
967     5.0                       3.0    2.0   2.0      2.0              3.0   
969     5.0                       3.0    2.0   1.0      2.0              3.0   
970     5.0                       3.0    5.0   1.0      4.0              5.0   
971     5.0                       4.0    2.0   4.0      3.0              4.0   
972     5.0                       3.0    3.0   3.0      3.0              3.0   
973     5.0                       3.0    2.0   5.0      5.0              4.0   
974     5.0                       3.0    1.0   2.0      2.0              2.0   
976     5.0                       3.0    4.0   2.0      3.0              3.0   
977     5.0                       4.0    4.0   2.0      2.0              1.0   
978     5.0                       2.0    2.0   1.0      2.0              2.0   
979     5.0                       3.0    4.0   2.0      2.0              4.0   
981     5.0                       4.0    1.0   2.0      4.0              5.0   
982     5.0                       4.0    2.0   1.0      1.0              3.0   
985     5.0                       2.0    1.0   3.0      2.0              4.0   
986     4.0                       3.0    2.0   5.0      3.0              5.0   
989     5.0                       5.0    3.0   2.0      2.0              4.0   
990     5.0                       4.0    2.0   1.0      1.0              2.0   
991   

In [61]:
young_music_full.to_csv("predicted_music_group_df.csv")

In [4]:
from sklearn.decomposition import PCA

In [14]:
pca = PCA(n_components=2)
pca.fit(X)

X2 = pca.transform(X)

In [23]:
X2[:,0]

array([-2.77977777e+00,  8.01301943e-01,  4.45725471e+00, -2.32048601e+00,
        3.55998956e+00, -3.58863719e+00, -2.16368989e+00,  2.80921755e+00,
       -1.90295337e+00, -9.44736898e-02,  9.17352667e-01, -3.23652911e+00,
        1.37184324e+00, -2.73140945e+00,  2.60244662e+00,  2.84620291e+00,
        4.34714047e+00,  1.81479532e+00, -6.81970183e-01,  1.78112107e+00,
       -4.67814776e+00,  2.00938689e+00, -2.90807234e+00,  6.47283565e-01,
        2.23196933e-01, -3.29194600e+00,  1.35135276e-01,  1.61524956e+00,
        1.74385825e+00, -1.46458284e+00, -1.52125829e+00, -1.51832849e+00,
        1.07542030e+00, -5.90331124e+00,  3.34389823e+00,  1.53162799e+00,
       -2.31111388e+00, -9.83489792e-01, -3.19884605e+00,  1.39635385e-01,
        2.84315693e+00,  4.03059104e+00,  1.39576358e+00,  3.67977544e+00,
        5.02699416e-01,  2.15603978e+00,  2.40070083e+00,  1.61158745e+00,
       -7.43686691e-01,  4.58433332e+00, -2.14107457e+00, -2.82028403e+00,
       -1.14102907e+00, -

In [24]:
young_music_full['X'] = X2[:,0]
young_music_full['Y'] = X2[:,1]

/Users/thalia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/thalia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
young_music_full

Music  Slow songs or fast songs  Dance  Folk  Country  Classical music  \
0       5.0                       3.0    2.0   1.0      2.0              2.0   
1       4.0                       4.0    2.0   1.0      1.0              1.0   
2       5.0                       5.0    2.0   2.0      3.0              4.0   
4       5.0                       3.0    4.0   3.0      2.0              4.0   
5       5.0                       3.0    2.0   3.0      2.0              3.0   
6       5.0                       5.0    5.0   3.0      1.0              2.0   
7       5.0                       3.0    3.0   2.0      1.0              2.0   
9       5.0                       3.0    2.0   5.0      2.0              2.0   
10      5.0                       3.0    3.0   2.0      1.0              2.0   
11      5.0                       3.0    1.0   1.0      1.0              4.0   
12      5.0                       3.0    1.0   2.0      1.0              4.0   
13      5.0                       3.0    5.0   3.0      2.0              1.0   
14      5.0                       3.0    2.0   1.0      1.0              2.0   
16      5.0                       3.0    3.0   1.0      1.0              1.0   
18      5.0                       3.0    5.0   4.0      3.0              4.0   
19      5.0                       4.0    3.0   3.0      2.0              4.0   
20      5.0                       3.0    3.0   2.0      3.0              4.0   
21      5.0                       5.0    1.0   1.0      3.0              2.0   
23      5.0                       3.0    4.0   2.0      2.0              2.0   
24      5.0                       2.0    3.0   1.0      1.0              4.0   
25      5.0                       3.0    4.0   2.0      1.0              2.0   
26      5.0                       5.0    5.0   5.0      4.0              5.0   
28      5.0                       3.0    5.0   1.0      1.0              1.0   
29      5.0                       4.0    3.0   4.0      2.0              3.0   
30      4.0                       3.0    4.0   3.0      3.0              3.0   
31      4.0                       3.0    4.0   1.0      3.0              2.0   
32      5.0                       5.0    3.0   1.0      3.0              2.0   
33      5.0                       4.0    2.0   2.0      3.0              4.0   
34      5.0                       4.0    3.0   2.0      1.0              3.0   
35      5.0                       3.0    3.0   3.0      1.0              4.0   
...     ...                       ...    ...   ...      ...              ...   
966     5.0                       5.0    1.0   1.0      1.0              5.0   
967     5.0                       3.0    2.0   2.0      2.0              3.0   
969     5.0                       3.0    2.0   1.0      2.0              3.0   
970     5.0                       3.0    5.0   1.0      4.0              5.0   
971     5.0                       4.0    2.0   4.0      3.0              4.0   
972     5.0                       3.0    3.0   3.0      3.0              3.0   
973     5.0                       3.0    2.0   5.0      5.0              4.0   
974     5.0                       3.0    1.0   2.0      2.0              2.0   
976     5.0                       3.0    4.0   2.0      3.0              3.0   
977     5.0                       4.0    4.0   2.0      2.0              1.0   
978     5.0                       2.0    2.0   1.0      2.0              2.0   
979     5.0                       3.0    4.0   2.0      2.0              4.0   
981     5.0                       4.0    1.0   2.0      4.0              5.0   
982     5.0                       4.0    2.0   1.0      1.0              3.0   
985     5.0                       2.0    1.0   3.0      2.0              4.0   
986     4.0                       3.0    2.0   5.0      3.0              5.0   
989     5.0                       5.0    3.0   2.0      2.0              4.0   
990     5.0                       4.0    2.0   1.0      1.0              2.0   
991   

In [142]:
defaultdict_sample = {"":0,"2.0":0,"3.0":0}
movie_df= young_music_full.groupby('cluster')['Movies'].value_counts().fillna(0)#.agg(lambda x: x.value_counts(sort=False).to_dict(into=defaultdict_sample))

In [146]:
movie_df.values
movie_df.index

MultiIndex(levels=[[0, 1, 2], [1.0, 2.0, 3.0, 4.0, 5.0]],
           codes=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2], [4, 3, 2, 1, 0, 4, 3, 2, 0, 4, 3, 2]],
           names=['cluster', 'Movies'])

In [154]:
a = pd.DataFrame(movie_df)
a.columns = ['count']
a['question'] = 'movie'
a.reset_index(inplace=True)
#[(0,4.0)]

In [171]:
a#[['cluster','question','Movies','count']]

cluster  Movies  count question
0         0     5.0    191    movie
1         0     4.0     50    movie
2         0     3.0     15    movie
3         0     2.0      3    movie
4         0     1.0      1    movie
5         1     5.0    256    movie
6         1     4.0     71    movie
7         1     3.0     28    movie
8         1     1.0      1    movie
9         2     5.0     43    movie
10        2     4.0     11    movie
11        2     3.0      4    movie

In [221]:
history_df= young_music_full.groupby('cluster')['History'].value_counts().unstack(level=1).fillna(0)
psychology_df= young_music_full.groupby('cluster')['Psychology'].value_counts().unstack(level=0).fillna(0)
movie_df= young_music_full.groupby('cluster')['Movies'].value_counts().unstack(level=1).fillna(0)

In [251]:
#new = pd.DataFrame(movie_df.unstack()).reset_index()
movie_df = movie_df.reset_index()
movie_df['quesiton']='movie'
movie_df['group'] = movie_df[]

Movies  cluster  1.0  2.0   3.0   4.0    5.0 quesiton
0             0  1.0  3.0  15.0  50.0  191.0    movie
1             1  1.0  0.0  28.0  71.0  256.0    movie
2             2  0.0  0.0   4.0  11.0   43.0    movie

colname = clustered_df.columns.name
colname + [1,2,3].to_string()
#clustered_df.columns = 
#clustered_df.columns.to_string() + colname 

In [ ]:
movie_df['question'] = 'movie'
movie_df
history_df['question'] = 'history'
psychology_df['question'] = 'psychology'

In [128]:
test = pd.DataFrame(movie_df)
test.columns = ['count']
test.reset_index(inplace=True)
#pd.melt(pd.DataFrame(movie_df), id_vars=['movie'], var_name='cluster', value_name='count')

list(np.repeat(range(3),5))*3
list(range(5))*3*3
[ele for ele in ['movie','history','psychology'] for i in range(15)] 

In [179]:
len(['movie','history','psychology'])
list(np.repeat(range(3),5))*len(['movie','history','psychology'])

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2]

In [242]:
pd.DataFrame(young_music_full['Movies'].value_counts())

Movies
5.0     490
4.0     132
3.0      47
2.0       3
1.0       2

In [244]:
def countsdf(df,questions,n_cluster):
    return_df = pd.DataFrame(columns = ['cluster','value','question','count'])#['cluster','question','value','count'])
    #return_df['cluster']=list(np.repeat(range(n_cluster),5))*len(questions)
    #return_df['value'] = list(range(5))*len(questions)*n_cluster
    #return_df['question'] = [ele for ele in questions for i in range(n_cluster*5)] 
    for question in questions:
        print(question)
        raw_count_series = df.groupby('cluster')[question].value_counts().unstack().fillna(0)
        raw_count_df = pd.DataFrame(raw_count_series.unstack()).reset_index()
        raw_count_df.columns = ['value','cluster','count']
        raw_count_df['question'] = question
        #print(raw_count_df)
        #return_df = pd.merge(return_df, raw_count_df,  how='left', left_on=['cluster','question','value'], right_on = ['cluster','question','value'])
        #return_df['count_x'].fillna(return_df['count_y'])
        return_df = return_df.append(raw_count_df,ignore_index=True)
        
        general_count = df[question].value_counts()
        
        print(raw_count_df)
        
        
    return(return_df)   
        
        
        

In [245]:
countsdf(young_music_full,['Movies','History','Psychology'],3)

Movies
    value  cluster  count question
0       0      1.0    1.0   Movies
1       0      2.0    3.0   Movies
2       0      3.0   15.0   Movies
3       0      4.0   50.0   Movies
4       0      5.0  191.0   Movies
5       1      1.0    1.0   Movies
6       1      2.0    0.0   Movies
7       1      3.0   28.0   Movies
8       1      4.0   71.0   Movies
9       1      5.0  256.0   Movies
10      2      1.0    0.0   Movies
11      2      2.0    0.0   Movies
12      2      3.0    4.0   Movies
13      2      4.0   11.0   Movies
14      2      5.0   43.0   Movies
History
    value  cluster  count question
0       0      1.0     22  History
1       0      2.0     48  History
2       0      3.0     62  History
3       0      4.0     64  History
4       0      5.0     64  History
5       1      1.0     46  History
6       1      2.0     74  History
7       1      3.0    107  History
8       1      4.0     71  History
9       1      5.0     58  History
10      2      1.0      1  History
11   

cluster  count    question value
0       1.0    1.0      Movies     0
1       2.0    3.0      Movies     0
2       3.0   15.0      Movies     0
3       4.0   50.0      Movies     0
4       5.0  191.0      Movies     0
5       1.0    1.0      Movies     1
6       2.0    0.0      Movies     1
7       3.0   28.0      Movies     1
8       4.0   71.0      Movies     1
9       5.0  256.0      Movies     1
10      1.0    0.0      Movies     2
11      2.0    0.0      Movies     2
12      3.0    4.0      Movies     2
13      4.0   11.0      Movies     2
14      5.0   43.0      Movies     2
15      1.0   22.0     History     0
16      2.0   48.0     History     0
17      3.0   62.0     History     0
18      4.0   64.0     History     0
19      5.0   64.0     History     0
20      1.0   46.0     History     1
21      2.0   74.0     History     1
22      3.0  107.0     History     1
23      4.0   71.0     History     1
24      5.0   58.0     History     1
25      1.0    1.0     History     2
26      2.0    9.0     History     2
27      3.0   21.0     History     2
28      4.0   10.0     History     2
29      5.0   17.0     History     2
30      1.0   28.0  Psychology     0
31      2.0   58.0  Psychology     0
32      3.0   60.0  Psychology     0
33      4.0   66.0  Psychology     0
34      5.0   48.0  Psychology     0
35      1.0   41.0  Psychology     1
36      2.0   84.0  Psychology     1
37      3.0  103.0  Psychology     1
38      4.0   77.0  Psychology     1
39      5.0   51.0  Psychology     1
40      1.0    3.0  Psychology     2
41      2.0   10.0  Psychology     2
42      3.0   11.0  Psychology     2
43      4.0   19.0  Psychology     2
44      5.0   15.0  Psychology     2

In [266]:
def countsdf(df,questions,n_cluster):
    return_df = pd.DataFrame(columns = ['cluster','question'])#['cluster','question','value','count'])
    #return_df['cluster']=list(np.repeat(range(n_cluster),5))*len(questions)
    #return_df['value'] = list(range(5))*len(questions)*n_cluster
    #return_df['question'] = [ele for ele in questions for i in range(n_cluster*5)] 
    for question in questions:
        print(question)
        raw_count_series = df.groupby('cluster')[question].value_counts().unstack().fillna(0)
        raw_count_df = raw_count_series.reset_index()
        raw_count_df['question'] = question
        #print(raw_count_df)
        #return_df = pd.merge(return_df, raw_count_df,  how='left', left_on=['cluster','question','value'], right_on = ['cluster','question','value'])
        #return_df['count_x'].fillna(return_df['count_y'])
        return_df = return_df.append(raw_count_df,ignore_index=True)
        
        general_count = pd.DataFrame(df[question].value_counts()).T
        print(general_count)#.unstack().fillna(0)
        general_count['cluster'] = 'all'
        general_count['question'] = question
        
        return_df = return_df.append(general_count,ignore_index=True)
        
        
        print(raw_count_df)
        
        
    return(return_df)  

In [267]:
countsdf(young_music_full,['Movies','History','Psychology'],3)

Movies
        5.0  4.0  3.0  2.0  1.0
Movies  490  132   47    3    2
Movies  cluster  1.0  2.0   3.0   4.0    5.0 question
0             0  1.0  3.0  15.0  50.0  191.0   Movies
1             1  1.0  0.0  28.0  71.0  256.0   Movies
2             2  0.0  0.0   4.0  11.0   43.0   Movies
History
         3.0  4.0  5.0  2.0  1.0
History  190  145  139  131   69
History  cluster  1.0  2.0  3.0  4.0  5.0 question
0              0   22   48   62   64   64  History
1              1   46   74  107   71   58  History
2              2    1    9   21   10   17  History
Psychology
            3.0  4.0  2.0  5.0  1.0
Psychology  174  162  152  114   72
Psychology  cluster  1.0  2.0  3.0  4.0  5.0    question
0                 0   28   58   60   66   48  Psychology
1                 1   41   84  103   77   51  Psychology
2                 2    3   10   11   19   15  Psychology


cluster    question   1.0    2.0    3.0    4.0    5.0
0        0      Movies   1.0    3.0   15.0   50.0  191.0
1        1      Movies   1.0    0.0   28.0   71.0  256.0
2        2      Movies   0.0    0.0    4.0   11.0   43.0
3      all      Movies   2.0    3.0   47.0  132.0  490.0
4        0     History  22.0   48.0   62.0   64.0   64.0
5        1     History  46.0   74.0  107.0   71.0   58.0
6        2     History   1.0    9.0   21.0   10.0   17.0
7      all     History  69.0  131.0  190.0  145.0  139.0
8        0  Psychology  28.0   58.0   60.0   66.0   48.0
9        1  Psychology  41.0   84.0  103.0   77.0   51.0
10       2  Psychology   3.0   10.0   11.0   19.0   15.0
11     all  Psychology  72.0  152.0  174.0  162.0  114.0

In [94]:
clustered_df.rename( columns=lambda s: 'Movie'+str(s))
clustered_df = clustered_df.add_prefix('Movie_')

In [ ]:
from collections import OrderedDict, defaultdict

In [104]:
movie_df.to_dict()

{0: {1.0: 1.0, 2.0: 3.0, 3.0: 15.0, 4.0: 50.0, 5.0: 191.0},
 1: {1.0: 1.0, 2.0: 0.0, 3.0: 28.0, 4.0: 71.0, 5.0: 256.0},
 2: {1.0: 0.0, 2.0: 0.0, 3.0: 4.0, 4.0: 11.0, 5.0: 43.0}}

In [109]:
grouped_dict = {}
for key in movie_df:
    grouped_dict[key] = {'movie':movie_df[key].to_dict(),'history':history_df[key].to_dict(),'pscychology':pscychology_df[key].to_dict()}

In [111]:
pd.DataFrame(grouped_dict)

0  \
history          {1.0: 22, 2.0: 48, 3.0: 62, 4.0: 64, 5.0: 64}   
movie        {1.0: 1.0, 2.0: 3.0, 3.0: 15.0, 4.0: 50.0, 5.0...   
pscychology      {1.0: 28, 2.0: 58, 3.0: 60, 4.0: 66, 5.0: 48}   

                                                             1  \
history         {1.0: 46, 2.0: 74, 3.0: 107, 4.0: 71, 5.0: 58}   
movie        {1.0: 1.0, 2.0: 0.0, 3.0: 28.0, 4.0: 71.0, 5.0...   
pscychology     {1.0: 41, 2.0: 84, 3.0: 103, 4.0: 77, 5.0: 51}   

                                                             2  
history            {1.0: 1, 2.0: 9, 3.0: 21, 4.0: 10, 5.0: 17}  
movie        {1.0: 0.0, 2.0: 0.0, 3.0: 4.0, 4.0: 11.0, 5.0:...  
pscychology       {1.0: 3, 2.0: 10, 3.0: 11, 4.0: 19, 5.0: 15}